# 🚀 단일 모델 실험 - SOLAR/Polyglot-Ko LLM 파인튜닝
> PRD 계획에 따른 LLM 모델 LoRA 파인튜닝

**목표 성능**: ROUGE-F1 70-73

In [1]:
# 환경 설정
import sys
import os
from pathlib import Path

# 프로젝트 루트 경로 추가
notebook_dir = Path.cwd()
project_root = notebook_dir.parent.parent.parent  # 3번만 parent 사용!

# 다른 프로젝트 경로 제거하고 현재 프로젝트 경로만 추가
sys.path = [p for p in sys.path if 'computer-vision-competition' not in p]
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project Root: {project_root}")
print(f"Current Dir: {notebook_dir}")

# 필요한 라이브러리 임포트
import yaml
import pandas as pd
import torch
from datetime import datetime
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType
import wandb

# 커스텀 모듈 임포트
from src.logging.notebook_logger import NotebookLogger
from src.utils.gpu_optimization.team_gpu_check import check_gpu_tier

print("Libraries imported successfully!")

Project Root: /home/ieyeppo/AI_Lab/natural-language-processing-competition
Current Dir: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH
Libraries imported successfully!


In [2]:
# 설정 파일 로드
config_path = notebook_dir / 'configs' / 'config_single_model.yaml'

with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

# 모델 선택
current_model = config['current_model']
model_config = config['models'][current_model]

print(f"Selected Model: {model_config['name']}")
print(f"Using LoRA: {model_config['use_lora']}")
if model_config['use_lora']:
    print(f"  - LoRA r: {model_config['lora_r']}")
    print(f"  - LoRA alpha: {model_config['lora_alpha']}")

Selected Model: upstage/SOLAR-10.7B-Instruct-v1.0
Using LoRA: True
  - LoRA r: 16
  - LoRA alpha: 32


In [3]:
# 로그 디렉토리 생성
log_dir = Path(config['paths']['log_dir'])
print(f"Log Directory: {log_dir}")
log_dir.mkdir(parents=True, exist_ok=True)

# 타임스탬프 생성
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# 로거 초기화
log_file = log_dir / f'single_model_{current_model}_{timestamp}.log'
logger = NotebookLogger(
    log_path=str(log_file),
    print_also=True
)

logger.write('='*50)
logger.write(f'Single Model Experiment: {current_model}')
logger.write(f'Timestamp: {timestamp}')
logger.write(f'Model: {model_config["name"]}')
logger.write('='*50)

Log Directory: logs/single_model
Single Model Experiment: solar
Timestamp: 20251010_090241
Model: upstage/SOLAR-10.7B-Instruct-v1.0


In [4]:
# GPU 체크
if torch.cuda.is_available():
    gpu_tier = check_gpu_tier()
    device = torch.device('cuda')
    logger.write(f"GPU: {torch.cuda.get_device_name(0)}")
    logger.write(f"GPU Tier: {gpu_tier}")
else:
    device = torch.device('cpu')
    logger.write("WARNING: No GPU available")

GPU: NVIDIA GeForce RTX 4090
GPU Tier: LOW


In [5]:
# LoRA 설정
if model_config['use_lora']:
    lora_config = LoraConfig(
        r=model_config['lora_r'],
        lora_alpha=model_config['lora_alpha'],
        lora_dropout=model_config['lora_dropout'],
        task_type=TaskType.CAUSAL_LM,
        target_modules=config['peft']['target_modules']
    )
    logger.write("LoRA configuration created")
    logger.write(f"  - r: {model_config['lora_r']}")
    logger.write(f"  - alpha: {model_config['lora_alpha']}")
    logger.write(f"  - dropout: {model_config['lora_dropout']}")

LoRA configuration created
  - r: 16
  - alpha: 32
  - dropout: 0.1


In [6]:
# 프롬프트 템플릿 확인
sample_dialogue = "#Person1#: 안녕하세요?\n#Person2#: 안녕하세요!"
prompt = config['prompt_template']['instruction_format'].format(
    dialogue=sample_dialogue,
    summary=""
)
logger.write("Prompt template loaded")
print("\nSample Prompt:")
print(prompt[:300] + "...")

Prompt template loaded

Sample Prompt:
### Instruction:
다음 대화를 3-5문장으로 요약해주세요. 핵심 내용과 중요한 정보를 포함시켜주세요.

### Input:
#Person1#: 안녕하세요?
#Person2#: 안녕하세요!

### Response:

...


In [7]:
# 데이터 경로 설정 및 로드
# config 파일의 경로 사용
def get_data_path(path_str):
    """config의 상대 경로를 절대 경로로 변환"""
    path = Path(path_str)
    if not path.is_absolute():
        path = notebook_dir / path
    return path

# config에서 데이터 경로 가져오기
train_path = get_data_path(config['paths']['train_file'])
dev_path = get_data_path(config['paths']['dev_file'])
test_path = get_data_path(config['paths']['test_file'])

logger.write(f"Loading data from config paths:")
logger.write(f"  - Train: {train_path}")
logger.write(f"  - Dev: {dev_path}")
logger.write(f"  - Test: {test_path}")

# 데이터 로드
train_df = pd.read_csv(train_path)
dev_df = pd.read_csv(dev_path)
test_df = pd.read_csv(test_path)

logger.write(f"Data loaded successfully!")
logger.write(f"Train samples: {len(train_df)}")
logger.write(f"Dev samples: {len(dev_df)}")
logger.write(f"Test samples: {len(test_df)}")

# 데이터 샘플 출력
print("\nSample train data:")
print(train_df[['fname', 'topic']].head(3))
print(f"\nDialogue sample (first 200 chars):")
print(train_df.iloc[0]['dialogue'][:200] + "...")

Loading data from config paths:
  - Train: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH/../../../data/raw/train.csv
  - Dev: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH/../../../data/raw/dev.csv
  - Test: /home/ieyeppo/AI_Lab/natural-language-processing-competition/notebooks/team/CHH/../../../data/raw/test.csv
Data loaded successfully!
Train samples: 12457
Dev samples: 499
Test samples: 499

Sample train data:
     fname  topic
0  train_0   건강검진
1  train_1  백신 접종
2  train_2  열쇠 분실

Dialogue sample (first 200 chars):
#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요? 
#Person2#: 건강검진을 받으려고 왔어요. 
#Person1#: 네, 5년 동안 검진을 안 받으셨네요. 매년 한 번씩 받으셔야 해요. 
#Person2#: 알죠. 특별히 아픈 데가 없으면 굳이 갈 필요가 없다고 생각했어요. 
#Person...


## LLM 파인튜닝 구현

실제 파인튜닝 코드는 config 설정에 따라 구현됩니다.